#  Neurocmbat TOP and StrokeMRI harmonized datasets

Note this must be run in the `neuron` environment

## import libraries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np


sys.path.insert(0, '../../')  # path to functions
import cvasl.vendor.neurocombat.neurocombat as neurocombat
import cvasl.harmony as har

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# filepath = '../open_work/internal_results/cleaned_pvc2s/' 
# filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
# filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 

In [ ]:
#pd.read_csv(filename_mri)# Datasets for this work
EDIS_path = '../our_datasets/EDIS/'
HELIUS_path = '../our_datasets/HELIUS/'
Insight46_path = '../our_datasets/Insight46/'
SABRE_path = '../our_datasets/SABRE/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
#HELIUS_file = os.path.join(HELIUS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)

# EDIS = pd.read_csv(EDIS_file)
# #HELIUS = pd.read_csv(HELIUS_file)
# Insight46 = pd.read_csv(Insight46_file)
# SABRE = pd.read_csv(SABRE_file)
# TOP = pd.read_csv(TOP_file)
# MRI = pd.read_csv(MRI_file)

In [ ]:
pd.read_csv(MRI_file)

In [ ]:
#filename_mri

In [ ]:
# below must be modified to get rid of columns we don't use
#STOP

In [ ]:
def prep_for_neurocombat(dataframename1, dataframename2):
    """
    This function takes two dataframes in the cvasl format,
    then turns them into three items needed for the 
    neurocombat algorithm with re-identification.
    
    """
    our_1_frame = pd.read_csv(dataframename1)
    our_2_frame = pd.read_csv(dataframename2)
    our_1_frame.columns = our_1_frame.columns.str.lower()
    our_2_frame.columns = our_2_frame.columns.str.lower()
    two_selection = our_2_frame.drop(['unnamed: 0'],axis=1)
    one_selection = our_1_frame.drop(['unnamed: 0'],axis=1)
    one_selection = one_selection.set_index('participant_id')
    two_selection = two_selection.set_index('participant_id')
    one_selection = one_selection.T
    two_selection = two_selection.T
    both_togetherF = pd.concat([one_selection, two_selection], axis=1, join="inner")
    print("Nan count",both_togetherF.isna().sum().sum())
    features_only = both_togetherF[2:]
    dictionary_features_len = len(features_only.T.columns)
    number = 0
    made_keys = [] 
    made_vals = [] 
    for n in features_only.T.columns:

        made_keys.append(number)
        made_vals.append(n)
        number +=1
    feature_dictF = dict(map(lambda i,j : (i,j) , made_keys,made_vals))
    ftF = features_only.reset_index()
    ftF = ftF.rename(columns={"index": "A"})
    ftF = ftF.drop(['A'], axis=1)
    ftF = ftF.dropna()
    btF = both_togetherF.reset_index()
    btF = btF.rename(columns={"index": "A"})
    btF = btF.drop(['A'], axis=1)
    btF = btF.dropna()
    len1 = len(one_selection.columns)
    len2 = len(two_selection.columns)
    return both_togetherF, ftF, btF, feature_dictF, len1, len2

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = prep_for_neurocombat(MRI_file, TOP_file)

In [ ]:
our_mri_data = pd.read_csv(MRI_file)
our_top_data = pd.read_csv(TOP_file)

In [ ]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [ ]:
# # make and save of csv of features only
# features_only = both_together[2:]
# #features_only.to_csv('features_only_top_mri.csv')

In [ ]:
ftF.to_csv('ftF_top_mri.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_togetherF.loc['sex',:].values.tolist(),
           'age':both_togetherF.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['age']
#Harmonization step:
data_combat = neurocombat.neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

# (n_dat, covars, batch_col, continuous_cols=continuous_cols,
#                                     categorical_cols=categorical_cols)

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
def make_topper(btF, row0,row1):
    """
    This function makes top rows for something harmonized
    out of the btF part produced with prep_for_neurocombat(dataframename1, dataframename2)
    """
    topperF = btF.head(2)
    topperF = topperF.rename_axis(None, axis="columns")
    topperF = topperF.reset_index(drop=False)
    topperF = topperF.rename(columns={"index": "char"})
    topperF['char'][0] = row0#'age'
    topperF['char'][1] = row1#'sex'
    return topperF

In [ ]:
topperF = make_topper(btF,'age', 'sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
neuro_harm_top =back_together.tail(len2)
neuro_harm_mri =back_together.head(len1)

In [ ]:
neuro_harm_top = neuro_harm_top.rename(feature_dictF, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_mri = neuro_harm_mri.reset_index()
neuro_harm_mri = neuro_harm_mri.rename(columns={"index": "participant_id"})
neuro_harm_mri

Save off to harmonized csv files

In [ ]:
neuro_harm_top = neuro_harm_top.reset_index()
neuro_harm_top = neuro_harm_top.rename(columns={"index": "participant_id"})
neuro_harm_top

In [ ]:
neuro_harm_mri.to_csv('neuro_harm_mri.csv')
neuro_harm_top.to_csv('neuro_harm_top.csv')

In [ ]:
STOP

merge behaviour needs evaluation and rewrite

In [ ]:
mri_selection = our_mri_data
#mri_selection.columns = mri_selection.columns.str.lower()
rename_dict = {
    'age': 'age_unharm', # will be the key on merge
    'sex':'sex_unharm',
    'gm_vol' :'gm_vol_unharm',
    'wm_vol' :'wm_vol_unharm',
    'csf_vol':'csf_vol_unharm',
    'gm_ivc_ratio': 'gm_ivc_ratio_unharm',
    'gmwm_ivc_ratio': 'gmwm_ivc_ratio_unharm',
    'wmh_vol' :'wmh_vol_unharm',
    'wmh_count' : 'wmh_count_unharm',
    'deepwm_b_cov':'deepwm_b_cov_unharm',
    'aca_b_cov': 'aca_b_cov_unharm',
    'mca_b_cov': 'mca_b_cov_unharm',
    'pca_b_cov': 'pca_b_cov_unharm',
    'totalgm_b_cov':'totalgm_b_cov_unharm',
    'deepwm_b':'deepwm_b_unharm',
    'aca_b':'aca_b_unharm',
    'mca_b':'mca_b_unharm',
    'pca_b':'pca_b_unharm',
    'totalgm_b':'totalgm_b_unharm', 

}
mri_selection = mri_selection.rename(columns = rename_dict)
mri_selection = mri_selection.merge(neuro_harm_mri, how= 'inner',)
mri_selection

In [ ]:
#neuro_harm_top.head(3)
our_mri_data

In [ ]:
top_selection = our_top_data
top_selection.columns = top_selection.columns.str.lower()
top_selection = top_selection.rename(columns = rename_dict)
top_selection = top_selection.merge(neuro_harm_top, how= 'inner',)
top_selection.columns

In [ ]:
top_selection.to_csv('top_before_after_neurocombat2way.csv')
mri_selection.to_csv('mri_before_after_neurocombat2way.csv')

In [ ]:
top_selection

Here we will also save off a version without white matter columns...optional

In [ ]:
# neuro_harm_mri_less = neuro_harm_mri.drop(columns=['wmh_vol', 'wmh_count'])
# neuro_harm_top_less = neuro_harm_top.drop(columns=['wmh_vol', 'wmh_count'])
# neuro_harm_mri_less.to_csv('less_neuro_harm_mri_mon.csv')
# neuro_harm_top_less.to_csv('less_neuro_harm_top_mon.csv')

# Investigate neurocombat results

In [ ]:
neuro_harm_mri.head(3) 

## We see neuroharmoney added negative numbers 


## let's see how many negative numbers we have

In [ ]:
number_columns = ['gm_vol',
                  'wm_vol',
                  'csf_vol',
                  'gm_ivc_ratio', 
                  'gmwm_ivc_ratio',
                  'wmh_vol',
                  'wmh_count',
                  'deepwm_b_cov',
                  'aca_b_cov',
                  'mca_b_cov',
                  'pca_b_cov',
                  'totalgm_b_cov',
                  'deepwm_b',
                  'aca_b',
                  'mca_b',
                  'pca_b',
                  'totalgm_b',]

In [ ]:
(neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
#our_top_data.head(3)
big_top = our_top_data.rename(columns={"Unnamed: 0": "level_0"})
big_top.head(3)

In [ ]:
new_harm_top = neuro_harm_top.reset_index()
new_harm_top = new_harm_top.reset_index()
new_harm_top = new_harm_top.rename(columns={"index": "participant_id"})
new_harm_top.head(3) 

In [ ]:
# difference in harmonized and original - raw
raw_differences = big_top[number_columns] - new_harm_top[number_columns]

In [ ]:
# difference in harmonized and original - raw
differences = (big_top[number_columns] - new_harm_top[number_columns])/big_top[number_columns]
differences

In [ ]:
differences_n = differences.apply(pd.to_numeric) #

In [ ]:
differences_n.describe()

## Now we can use the versions of TOP and StrokeMRI with log base 10 revision of white matter hyperintensity count, as well as white matter hyperintensity  volume

In [ ]:
log_filepath = '../open_work/internal_results/loggy/' 
# this needs to be redone, due to bad column
log_filename_mri = os.path.join(log_filepath,'stroke_loged_mon.csv') 
log_filename_top = os.path.join(log_filepath,'top_loged_mon.csv') 

In [ ]:
#log_mri_data = pd.read_csv(log_filename_mri)
#log_top_data = pd.read_csv(log_filename_top)

In [ ]:
log_both_togetherF, log_ftF, log_btF, log_feature_dictF, log_len1, log_len2 = prep_for_neurocombat(
    log_filename_mri,
    log_filename_top)

In [ ]:
# # make and save of csv of features only
# log_features_only = log_both_together[2:]
# #log_features_only.to_csv('log_features_only_top_mri.csv')

In [ ]:
log_ftF.to_csv('log_ftF_top_mri.csv')

In [ ]:
log_data = np.genfromtxt('log_ftF_top_mri.csv', delimiter=",", skip_header=1)
log_data = log_data[:, 1:]

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * log_len1
last_columns_as_two = [2] * log_len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':log_both_togetherF.loc['sex',:].values.tolist(),
           'age':log_both_togetherF.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'

#Harmonization step:
log_data_combat = neuroCombat(dat=log_data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

In [ ]:
log_data_combat

In [ ]:
log_neurocombat = pd.DataFrame(log_data_combat)
#log_neurocombat

In [ ]:
log_topperF = make_topper(log_btF,'age', 'sex')

In [ ]:
log_bottom = log_neurocombat.reset_index(drop=False)
log_bottom = log_bottom.rename(columns={"index": "char"})
log_bottom.columns = log_topperF.columns
#log_bottom

In [ ]:
log_back_together = pd.concat([log_topperF, log_bottom])
log_back_together = log_back_together.T
#log_back_together

In [ ]:
log_new_header = log_back_together.iloc[0] #grab the first row for the header
log_back_together.columns = new_header #set the header row as the df header
log_back_together = log_back_together[1:]
#log_back_together

In [ ]:
log_back_together.head(514).tail(5)

In [ ]:
log_back_together.tail(527).head(5)

In [ ]:
log_neuro_harm_top =log_back_together.tail(log_len2)
log_neuro_harm_mri =log_back_together.head(log_len1)

In [ ]:
log_neuro_harm_top = log_neuro_harm_top.rename(log_feature_dictF, axis='columns')
log_neuro_harm_mri = log_neuro_harm_mri.rename(log_feature_dictF, axis='columns')

In [ ]:
(log_neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
## There are still negative numbers in the outcome...but there is a difference

In [ ]:
# log_neuro_harm_mri.to_csv('log_neuro_harm_mri_mon.csv')
# log_neuro_harm_top.to_csv('log_neuro_harm_top_mon.csv')

In [ ]:
log_neuro_harm_mri.head(3)

In [ ]:
neuro_harm_mri.head(3)

In [ ]:
dif_log_to_reg = log_neuro_harm_mri[number_columns] - neuro_harm_mri[number_columns]
dif_log_to_reg 

In [ ]:
dif_log_to_reg.sum().sum()

# So changing the two columns to theirlog made la difference in the harmarmonized outcomes...now let's look at how this plays out when we make models...